In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import panel as pn
pn.extension('plotly')

# Ajuste de display opcional (descomente se preferir)
# pd.set_option('display.max_columns', 200)
# pd.set_option('display.width', 200)

In [ ]:
# Carregar dados
GAME_CSV = 'csv/game.csv'
LINE_CSV = 'csv/line_score.csv'

game = pd.read_csv(GAME_CSV, parse_dates=['game_date'], low_memory=False)
line = pd.read_csv(LINE_CSV, parse_dates=['game_date_est'], low_memory=False)

# Converter colunas importantes para numérico (forçando coercion onde houver strings vazias)
num_cols = ['pts_home','pts_away','ast_home','ast_away','fgm_home','fga_home','fgm_away','fga_away']
for c in num_cols:
    if c in game.columns:
        game[c] = pd.to_numeric(game[c], errors='coerce')
# Também garantir no line (pts em line_score)
if 'pts_home' in line.columns:
    line['pts_home'] = pd.to_numeric(line['pts_home'], errors='coerce')
if 'pts_away' in line.columns:
    line['pts_away'] = pd.to_numeric(line['pts_away'], errors='coerce')

# 1) Métricas principais
total_games = game['game_id'].nunique()
total_assists = game['ast_home'].sum(skipna=True) + game['ast_away'].sum(skipna=True)
total_points = game['pts_home'].sum(skipna=True) + game['pts_away'].sum(skipna=True)
# Média de pontos por jogo por time (média histórica por time por partida)
avg_points_per_team_game = ((game['pts_home'] + game['pts_away']) / 2.0).mean()
# Percentual de arremessos convertidos (FGM / FGA) somando home + away
fg_made = game['fgm_home'].sum(skipna=True) + game['fgm_away'].sum(skipna=True)
fg_att = game['fga_home'].sum(skipna=True) + game['fga_away'].sum(skipna=True)
fg_pct = fg_made / fg_att if fg_att and not np.isnan(fg_att) else np.nan

# Formatação simples para exibição (PT-BR style: ',' decimal)
def br_num(x, decimals=2):
    if pd.isna(x):
        return '-'
    s = f'{x:,.{decimals}f}'
    # trocar ponto decimal por vírgula e manter separador de milhar como '.'
    s = s.replace(',', 'X').replace('.', ',').replace('X', '.')
    return s

kpis = {
    'Média de Pontos/Jogo (por time)': br_num(avg_points_per_team_game, 2),
    'Total de Assistências': br_num(total_assists, 0),
    'Total de Jogos': br_num(total_games, 0),
    'Percentual Arremessos Convertidos (FG%)': (br_num(fg_pct * 100.0, 2) + '%') if not pd.isna(fg_pct) else '-',
    'Total de Pontos': br_num(total_points, 0),
}

# Mostrar KPIs (a célula pode ser editada para exibir de forma diferente)
# kpis

{'Média de Pontos/Jogo (por time)': '102,81',
 'Total de Assistências': '2.298.979',
 'Total de Jogos': '65.642',
 'Percentual Arremessos Convertidos (FG%)': '60,79%',
 'Total de Pontos': '13.508.212'}

In [ ]:
# 2) Preparar dados por ano (usando line_score que traz pts_home / pts_away e a data)
line_clean = line.copy()
line_clean['year'] = pd.to_datetime(line_clean['game_date_est'], errors='coerce').dt.year
by_year = line_clean.groupby('year', dropna=True).agg({'pts_home':'sum','pts_away':'sum'}).reset_index().sort_values('year')
by_year['total'] = by_year['pts_home'] + by_year['pts_away']
# by_year.head()

,year,pts_home,pts_away,total
0,1946,6709.0,6627.0,13336.0
1,1947,16809.0,16634.0,33443.0
2,1948,18617.0,18425.0,37042.0
3,1949,32785.0,32274.0,65059.0
4,1950,34386.0,34362.0,68748.0


In [ ]:
# 3) Gráfico: barras empilhadas (pontos casa / pontos fora) + linha do total
fig = go.Figure()
fig.add_trace(go.Bar(name='Pontos em Casa', x=by_year['year'], y=by_year['pts_home'], marker_color='#1f77b4'))
fig.add_trace(go.Bar(name='Pontos Fora', x=by_year['year'], y=by_year['pts_away'], marker_color='#9467bd'))
fig.add_trace(go.Scatter(name='Total Pontos', x=by_year['year'], y=by_year['total'], mode='lines+markers', marker_color='#2ca02c'))
fig.update_layout(barmode='stack', title_text='Total de Pontos por Ano (Casa vs Fora)', xaxis_title='Ano', yaxis_title='Pontos', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), template='plotly_dark', height=450)
fig.update_yaxes(tickformat=',')
# fig

In [ ]:
# 4) Montar layout interativo com Panel: filtro por time + KPIs + gráfico
# Preparar lista de times (nomes completos) a partir do dataframe 'game'
teams = sorted(pd.unique(game['team_name_home'].dropna().tolist() + game['team_name_away'].dropna().tolist()))

# Widget: seleção múltipla de times (vazio = todos)
team_sel = pn.widgets.MultiSelect(name='Filtrar por Time (vazio = todos)', options=teams, size=8)

# Para mapear nomes por jogo no line_score, fazemos merge com game (para garantir nomes completos)
# Certificar que line_clean existe (criado na célula anterior)
line_merged = line_clean.merge(game[['game_id','team_name_home','team_name_away']], on='game_id', how='left')

def make_dashboard(selected_teams):
    # Se nada selecionado -> mostrar todos (usar valores pré-calculados)
    if not selected_teams:
        kpis_local = kpis
        df_by_year = by_year.copy()
    else:
        sel = set(selected_teams)
        # Filtrar jogos onde o time aparece como home ou away
        mask_home = game['team_name_home'].isin(sel)
        mask_away = game['team_name_away'].isin(sel)
        games_sel = game[mask_home | mask_away]
        # Métricas por time(s)
        team_games = games_sel['game_id'].nunique()
        team_pts = games_sel.loc[mask_home, 'pts_home'].sum(skipna=True) + games_sel.loc[mask_away, 'pts_away'].sum(skipna=True)
        team_assists = games_sel.loc[mask_home, 'ast_home'].sum(skipna=True) + games_sel.loc[mask_away, 'ast_away'].sum(skipna=True)
        team_fgm = games_sel.loc[mask_home, 'fgm_home'].sum(skipna=True) + games_sel.loc[mask_away, 'fgm_away'].sum(skipna=True)
        team_fga = games_sel.loc[mask_home, 'fga_home'].sum(skipna=True) + games_sel.loc[mask_away, 'fga_away'].sum(skipna=True)
        team_fg_pct = team_fgm / team_fga if team_fga and not np.isnan(team_fga) else np.nan
        team_avg_pts = team_pts / team_games if team_games else np.nan
        kpis_local = {
            'Média de Pontos/Jogo (por time)': br_num(team_avg_pts, 2),
            'Total de Assistências': br_num(team_assists, 0),
            'Total de Jogos': br_num(team_games, 0),
            'Percentual Arremessos Convertidos (FG%)': (br_num(team_fg_pct * 100.0, 2) + '%') if not pd.isna(team_fg_pct) else '-',
            'Total de Pontos': br_num(team_pts, 0),
        }
        # Agrupar por ano: pontos em casa quando time é home, pontos fora quando time é away
        pts_home_by_year = line_merged[line_merged['team_name_home'].isin(sel)].groupby('year')['pts_home'].sum()
        pts_away_by_year = line_merged[line_merged['team_name_away'].isin(sel)].groupby('year')['pts_away'].sum()
        years = sorted(set(pts_home_by_year.index).union(set(pts_away_by_year.index)))
        df_by_year = pd.DataFrame({'year': years}).set_index('year')
        df_by_year['pts_home'] = pts_home_by_year.reindex(years, fill_value=0)
        df_by_year['pts_away'] = pts_away_by_year.reindex(years, fill_value=0)
        df_by_year = df_by_year.reset_index()
        df_by_year['total'] = df_by_year['pts_home'] + df_by_year['pts_away']
    # Criar figura a partir de df_by_year
    fig_local = go.Figure()
    fig_local.add_trace(go.Bar(name='Pontos em Casa', x=df_by_year['year'], y=df_by_year['pts_home'], marker_color='#1f77b4'))
    fig_local.add_trace(go.Bar(name='Pontos Fora', x=df_by_year['year'], y=df_by_year['pts_away'], marker_color='#9467bd'))
    fig_local.add_trace(go.Scatter(name='Total Pontos', x=df_by_year['year'], y=df_by_year['total'], mode='lines+markers', marker_color='#2ca02c'))
    fig_local.update_layout(barmode='stack', title_text='Total de Pontos por Ano (Casa vs Fora)', xaxis_title='Ano', yaxis_title='Pontos', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), template='plotly_dark', height=450)
    fig_local.update_yaxes(tickformat=',')
    # Montar cartões KPI
    cards_local = []
    for title, value in kpis_local.items():
        md_html = f"""
        <div style="background:#0f1720;padding:12px;border-radius:8px;color:#dfefff;width:240px;height:140px;display:flex;flex-direction:column;justify-content:center;align-items:center">
          <strong>{title}</strong>
          <h1 style="margin:4px 0 0 0">{value}</h1>
        </div>
        """
        cards_local.append(pn.pane.HTML(md_html, width=240))
    kpi_row_local = pn.Row(pn.Row(team_sel), *cards_local, sizing_mode='stretch_width')
    chart_local = pn.pane.Plotly(fig_local, config={'displayModeBar': False}, sizing_mode='stretch_width')
    return pn.Column(pn.pane.Markdown('## Visão Geral'), kpi_row_local, pn.Spacer(height=10), chart_local)

# Usar pn.bind para criar um painel reativo que atualiza quando o widget muda
dashboard_reactive = pn.bind(make_dashboard, team_sel)

# Layout final com o seletor de times acima
dashboard_app = pn.Column(dashboard_reactive)

# Exibir o app (no Jupyter a saída é renderizada; para servir use 'panel serve')
dashboard_app